In [85]:
from array_variables import location_data, categories, locations_in_file, each_location, fastest_categories, fastest_coordinates, full_coordinate_array, num_per_category
from files_pre_conversion.reuseable_gpc_variables import dr, dl, du, dd, plus, min, ab, yb, bb, xb, zr, zl, rb, lb, tab, get_val, set_val, wait, combo_stop, combo_run, set_led, prnt, event_press
from files_pre_conversion.reusable_gpc_functions import string_to_variable, function_firstline, end_block, write_command, generate_if, generate_else

In [106]:
file = open("auto_teleport_loop.gpc", "w")

wait_time = [100]

category = 0

location = 0

program_name = r"Menu Fast Travel\nAll Locations"

instruction_page = r"Set Destination\nCategory: UP/DOWN\nLocation: Left/Right"

num_tabs = 1

In [107]:
# common_params = {"file":file, "num_tabs":num_tabs}

In [108]:
def assign_variable(name, value, num_tabs = 0):
    tab = '\t'
    file.write(f"{num_tabs*tab}{name} = {value};\n")

In [109]:
def declare_variable(type, name, value):
    file.write(f"{type} ")
    assign_variable(name, value)

In [110]:
def declare_array(array, array_string = ''):
    array_string += r'{'
    for item in array[:-1]:
        array_string += f'{item},'
    array_string += f'{array[-1]}' + r'}'
    return array_string

In [111]:
def declare_string(var_name, target_string):
    variable_name = string_to_variable(var_name)
    file.write(f'const string {variable_name} = "{target_string}";\n')

In [112]:
def declare_strings_from_array(string_array):
    for item in string_array:
        declare_string(item, item)

In [113]:
def modify_variable_value(current_variable, added_value, num_tabs = 1):
    tab = "\t"
    file.write(f'{tab * num_tabs}{current_variable} = {current_variable} + {added_value};\n')

In [114]:
def generate_if_event_press(button, modified_variable, added_value, num_tabs, file):
    generate_if(f"(event_press({button}))", num_tabs, file)
    num_tabs += 1
    modify_variable_value(modified_variable, added_value, num_tabs)
    num_tabs -= 1
    end_block(num_tabs, file)

In [115]:
def generate_if_reassignment(variable_name, comparison, value_assigned, num_tabs):
    generate_if(f'{variable_name} {comparison}', num_tabs, file)
    assign_variable(variable_name, value_assigned, num_tabs + 1)
    write_command(wait, [100], num_tabs + 1, file)
    end_block(num_tabs, file)

In [116]:
def write_selections():
    num_tabs = 1
    file.write("function get_destination()" + r'{' + '\n')
    generate_if(f'get_val({zl})', num_tabs, file)
    num_tabs += 1
    num_tabs += 1
    generate_if_event_press(dd, "category", 1, num_tabs, file)
    generate_if_event_press(du, "category", -1, num_tabs, file)
    generate_if_event_press(dl, "location", -1, num_tabs, file)
    generate_if_event_press(dr, "location", 1, num_tabs, file)
    generate_if_reassignment("category", "< 0", 4, num_tabs)
    generate_if_reassignment("category", "> 4", 0, num_tabs)
    generate_if_reassignment('location', '< 0', 'num_per_category[category - 1]', num_tabs)
    generate_if_reassignment('location', '> num_per_category[category - 1]', 0, num_tabs)
    generate_if(f'event_press({rb})', num_tabs, file)
    assign_variable('SCRIPT_RUNNING', '!SCRIPT_RUNNING', num_tabs)
    end_block(num_tabs, file)
    end_block(num_tabs - 1, file)
    end_block(num_tabs - 2, file)

In [117]:
def generate_print(name, num_tabs, file):
    # write_command(x_value, y_value, font, color, string_address)
    write_command("print", [20, 20, "OLED_FONT_MEDIUM", "OLED_WHITE", name], num_tabs, file)

In [118]:
def set_coordinate_position(i, num_tabs, file):
    assign_variable("coordinate_position", i, num_tabs)



In [119]:
def set_coordinates(i, array, num_tabs, file):
    file.write('function set_coordinates()' + r"{" + "\n")
    num_tabs += 1
    assign_variable("category_coordinate", f"{array}[{i}][0]", num_tabs)
    assign_variable("x_coordinate", f"{array}[{i}][1]", num_tabs)
    assign_variable("y_coordinate", f"{array}[{i}][2]", num_tabs)
    end_block(num_tabs - 1, file)

In [120]:
def assignment_string(name, value):
    string = f'{name} = {value};\n'
    return string

In [121]:
def generate_statement_array(statement_array, num_tabs):
    for item in statement_array:
        new_string += f'{num_tabs * '\t'}{item}'
        new_string += '\t}\n'
    return new_string

In [122]:
def write_function_string(name, statement_array, arguments=''):
    new_string = f'function {name}({arguments})' + r'{' + '\n'

    return new_string


In [123]:
def display_selection(num_tabs, file):
    file.write('function display_selection()' + r'{' + '\n')
    tab = '\t'
    category = 1
    location_num = 0
    for i, location in enumerate(each_location):
        category_index = category - 1
        if num_per_category[category_index] < location_num:
            category += 1
            location_num = 0
        else:
            location_num += 1
        location_variable = string_to_variable(location)
        file.write(f'{tab * num_tabs}if (category == {category} & location == {location_num})' + r'{' + '\n')
        set_coordinate_position(i, num_tabs + 1, file)
        generate_print(location_variable, num_tabs + 1, file)
        end_block(num_tabs, file)
    end_block(num_tabs - 1, file)


In [124]:
def initialize_variables():
    declare_variable("int", "SCRIPT_RUNNING", "FALSE")
    declare_variable("int", "category", "0")
    declare_variable("int", "location", "0")
    declare_variable('int', 'category_coordinate', 0)
    declare_variable('int', 'x_coordinate', 0)
    declare_variable('int', 'y_coordinate', 0)
    declare_variable('int', 'coordinate_position', 0)
    declare_variable("const string", "location_name_in_use", '""')
    file.write(declare_array(num_per_category, "const int num_per_category[] = "))
    file.write(';\n')
    file.write(f"const int full_coordinate_array[][] =" + r'{')
    for coordinate_set in full_coordinate_array[:-1]:
        coordinate_string = declare_array(coordinate_set)
        file.write(coordinate_string + r', ')
    file.write(f'{declare_array(full_coordinate_array[-1])}' + r'};' + '\n')
    declare_string("instruction page", instruction_page)
    declare_strings_from_array(each_location)


In [125]:
initialize_variables()
write_selections()
display_selection(num_tabs, file)
set_coordinates('coordinate_position', 'full_coordinate_array', 0, file)